# Amazon Sentiment Analysis

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import re
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel

/opt/sw/spack/apps/linux-rhel8-x86_64_v2/gcc-10.3.0/python-3.9.9-jh/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2025-02-06 12:40:13.045187: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-06 12:40:13.086334: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-06 12:40:13.382387: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-06 12:40:13.564400: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to reg

In [2]:
train_file = "./dataset/train_data.csv"
test_file  = "./dataset/test_data.csv"
test_hidden_file = "./dataset/test_data_hidden.csv"

In [3]:
#Load Dataset
df_train = pd.read_csv(train_file)
df_test = pd.read_csv(test_file)
df_test_hidden = pd.read_csv(test_hidden_file)

In [4]:
train_info = df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               4000 non-null   object
 1   brand              4000 non-null   object
 2   categories         4000 non-null   object
 3   primaryCategories  4000 non-null   object
 4   reviews.date       4000 non-null   object
 5   reviews.text       4000 non-null   object
 6   reviews.title      3990 non-null   object
 7   sentiment          4000 non-null   object
dtypes: object(8)
memory usage: 250.1+ KB


In [5]:
test_info = df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               1000 non-null   object
 1   brand              1000 non-null   object
 2   categories         1000 non-null   object
 3   primaryCategories  1000 non-null   object
 4   reviews.date       1000 non-null   object
 5   reviews.text       1000 non-null   object
 6   reviews.title      997 non-null    object
dtypes: object(7)
memory usage: 54.8+ KB


In [6]:
test_hidden_info = df_test_hidden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               1000 non-null   object
 1   brand              1000 non-null   object
 2   categories         1000 non-null   object
 3   primaryCategories  1000 non-null   object
 4   reviews.date       1000 non-null   object
 5   reviews.text       1000 non-null   object
 6   reviews.title      997 non-null    object
 7   sentiment          1000 non-null   object
dtypes: object(8)
memory usage: 62.6+ KB


In [7]:
#printing first few rows


In [8]:
train_head = df_train.head()
train_head

,name,brand,categories,primaryCategories,reviews.date,reviews.text,reviews.title,sentiment
0,"All-New Fire HD 8 Tablet, 8"" HD Display, Wi-Fi...",Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",Electronics,2016-12-26T00:00:00.000Z,Purchased on Black FridayPros - Great Price (e...,Powerful tablet,Positive
1,Amazon - Echo Plus w/ Built-In Hub - Silver,Amazon,"Amazon Echo,Smart Home,Networking,Home & Tools...","Electronics,Hardware",2018-01-17T00:00:00.000Z,I purchased two Amazon in Echo Plus and two do...,Amazon Echo Plus AWESOME,Positive
2,Amazon Echo Show Alexa-enabled Bluetooth Speak...,Amazon,"Amazon Echo,Virtual Assistant Speakers,Electro...","Electronics,Hardware",2017-12-20T00:00:00.000Z,Just an average Alexa option. Does show a few ...,Average,Neutral
3,"Fire HD 10 Tablet, 10.1 HD Display, Wi-Fi, 16 ...",Amazon,"eBook Readers,Fire Tablets,Electronics Feature...","Office Supplies,Electronics",2017-08-04T00:00:00.000Z,"very good product. Exactly what I wanted, and ...",Greattttttt,Positive
4,"Brand New Amazon Kindle Fire 16gb 7"" Ips Displ...",Amazon,"Computers/Tablets & Networking,Tablets & eBook...",Electronics,2017-01-23T00:00:00.000Z,This is the 3rd one I've purchased. I've bough...,Very durable!,Positive


In [9]:
test_head = df_test.head()
test_head

,name,brand,categories,primaryCategories,reviews.date,reviews.text,reviews.title
0,"Fire Tablet, 7 Display, Wi-Fi, 16 GB - Include...",Amazon,"Fire Tablets,Computers/Tablets & Networking,Ta...",Electronics,2016-05-23T00:00:00.000Z,Amazon kindle fire has a lot of free app and c...,very handy device
1,Amazon Echo Show Alexa-enabled Bluetooth Speak...,Amazon,"Computers,Amazon Echo,Virtual Assistant Speake...","Electronics,Hardware",2018-01-02T00:00:00.000Z,The Echo Show is a great addition to the Amazo...,Another winner from Amazon
2,"All-New Fire HD 8 Tablet, 8"" HD Display, Wi-Fi...",Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",Electronics,2017-01-02T00:00:00.000Z,Great value from Best Buy. Bought at Christmas...,simple to use and reliable so far
3,"Brand New Amazon Kindle Fire 16gb 7"" Ips Displ...",Amazon,"Computers/Tablets & Networking,Tablets & eBook...",Electronics,2017-03-25T00:00:00.000Z,"I use mine for email, Facebook ,games and to g...",Love it!!!
4,Amazon Echo Show Alexa-enabled Bluetooth Speak...,Amazon,"Computers,Amazon Echo,Virtual Assistant Speake...","Electronics,Hardware",2017-11-15T00:00:00.000Z,This is a fantastic item & the person I bought...,Fantastic!


In [10]:
test_head_hidden = df_test_hidden.head()
test_head_hidden

,name,brand,categories,primaryCategories,reviews.date,reviews.text,reviews.title,sentiment
0,"Fire Tablet, 7 Display, Wi-Fi, 16 GB - Include...",Amazon,"Fire Tablets,Computers/Tablets & Networking,Ta...",Electronics,2016-05-23T00:00:00.000Z,Amazon kindle fire has a lot of free app and c...,very handy device,Positive
1,Amazon Echo Show Alexa-enabled Bluetooth Speak...,Amazon,"Computers,Amazon Echo,Virtual Assistant Speake...","Electronics,Hardware",2018-01-02T00:00:00.000Z,The Echo Show is a great addition to the Amazo...,Another winner from Amazon,Positive
2,"All-New Fire HD 8 Tablet, 8"" HD Display, Wi-Fi...",Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",Electronics,2017-01-02T00:00:00.000Z,Great value from Best Buy. Bought at Christmas...,simple to use and reliable so far,Positive
3,"Brand New Amazon Kindle Fire 16gb 7"" Ips Displ...",Amazon,"Computers/Tablets & Networking,Tablets & eBook...",Electronics,2017-03-25T00:00:00.000Z,"I use mine for email, Facebook ,games and to g...",Love it!!!,Positive
4,Amazon Echo Show Alexa-enabled Bluetooth Speak...,Amazon,"Computers,Amazon Echo,Virtual Assistant Speake...","Electronics,Hardware",2017-11-15T00:00:00.000Z,This is a fantastic item & the person I bought...,Fantastic!,Positive


 ### Data Preprocessing

In [11]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [12]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/gvaidya2/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gvaidya2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/gvaidya2/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [14]:
def preprocess(text):
    if pd.isna(text):
        return ""
    text=text.lower()
    text=re.sub(r'[^a-z\s]','',text)
    tokens= word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)

In [15]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /home/gvaidya2/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [16]:
df_train['cleaned_txt'] = df_train['reviews.text'].apply(preprocess)

In [17]:
df_train['cleaned_txt'].head()

0    purchased black fridaypros great price even sa...
1    purchased two amazon echo plus two dot plus fo...
2    average alexa option show thing screen still l...
3               good product exactly wanted good price
4    rd one ive purchased ive bought one niece case...
Name: cleaned_txt, dtype: object

In [18]:
df_test['cleaned_txt'] = df_test['reviews.text'].apply(preprocess)

In [19]:
#converting sentiment labels to numerical values

sentiment_mapping = {'Positive': +1, 'Negative': -1, 'Neutral': 0}
df_train['sentiment_label']=df_train['sentiment'].map(sentiment_mapping)
df_test_hidden['sentiment_label'] = df_test_hidden['sentiment'].map(sentiment_mapping)

#save the preprocessed file
preprocessed_filepath= "./dataset/train_preprocessed.csv"
df_train.to_csv(preprocessed_filepath,index=False)

print(f"Preprocessed data saved at: {preprocessed_filepath}")

Preprocessed data saved at: ./dataset/train_preprocessed.csv


### Feature Engineering

#### TF IDF

In [20]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
preprocessed_filepath= "./dataset/train_preprocessed.csv"
df_train = pd.read_csv(preprocessed_filepath)

In [22]:
tfidf_vectorizer =  TfidfVectorizer(max_features=5000)
tfidf_features = tfidf_vectorizer.fit_transform(df_train['cleaned_txt']) 

df_test['cleaned_text'] = df_test['reviews.text'].apply(preprocess)


In [23]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(df_train['cleaned_txt'])
X_test_tfidf = tfidf_vectorizer.transform(df_test['cleaned_txt'])
y_train = df_train['sentiment_label'].values
y_test_actual = df_test_hidden['sentiment_label'].values

In [24]:
# Train Logistic Regression on TF-IDF
tfidf_model = LogisticRegression(max_iter=1000)
tfidf_model.fit(X_train_tfidf, y_train)
y_pred_tfidf = tfidf_model.predict(X_test_tfidf)

#### Word2Vec Embeddings

In [25]:
from gensim.models import Word2Vec
tok_train = df_train['cleaned_txt'].apply(lambda x:x.split())
tok_test = df_test['cleaned_txt'].apply(lambda x:x.split())

w2v_model = Word2Vec(sentences=tok_train, vector_size=100, window=5, min_count=2, workers=4)

In [26]:
print(w2v_model)

Word2Vec<vocab=2612, vector_size=100, alpha=0.025>


In [27]:
def get_w2v_embedding(text):
    words = text.split()
    word_vec = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    return np.mean(word_vec,axis=0) if word_vec else np.zeros(100)

X_train_w2v = np.array(df_train['cleaned_txt'].apply(get_w2v_embedding).tolist())
X_test_w2v = np.array(df_test['cleaned_txt'].apply(get_w2v_embedding).tolist())


In [28]:
# print(word2vec_features)
# Train SVM on Word2Vec
w2v_model_svm = SVC(kernel='linear')
w2v_model_svm.fit(X_train_w2v, y_train)
y_pred_w2v = w2v_model_svm.predict(X_test_w2v)


#### Bert Embeddings

In [29]:
import torch
from transformers import BertTokenizer, BertModel

device = torch.device("cpu")
bert_tok= BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)


In [30]:
def get_bert_emb(text):
    tokens= bert_tok(text, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
    with torch.no_grad():
        outputs = bert_model(**tokens)
    return outputs.last_hidden_state[:,0,:].squeeze().numpy()

X_train_bert = np.array(df_train['cleaned_txt'].apply(get_bert_emb).tolist())
X_test_bert = np.array(df_test['cleaned_txt'].apply(get_bert_emb).tolist())


    

In [31]:
bert_model_logreg=LogisticRegression(max_iter=1000)
bert_model_logreg.fit(X_train_bert,y_train)
y_pred_bert = bert_model_logreg.predict(X_test_bert)

#### Model Evaluation

In [33]:
!pip install ace_tools

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [38]:
import pickle

# Save TF-IDF Model & Vectorizer
with open("sentiment_model.pkl", "wb") as model_file:
    pickle.dump(tfidf_model, model_file)

with open("tfidf_vectorizer.pkl", "wb") as vec_file:
    pickle.dump(tfidf_vectorizer, vec_file)

# Save Word2Vec Model & SVM Classifier
w2v_model.save("w2v_model.pkl")

with open("w2v_svm.pkl", "wb") as model_file:
    pickle.dump(w2v_model_svm, model_file)

# Save BERT Model
with open("bert_model.pkl", "wb") as model_file:
    pickle.dump(bert_model_logreg, model_file)


In [43]:
train_results = [
    ("TF-IDF (Logistic Regression)", accuracy_score(y_train, tfidf_model.predict(X_train_tfidf)) * 100),
    ("Word2Vec (SVM)", accuracy_score(y_train, w2v_model_svm.predict(X_train_w2v)) * 100),
    ("BERT (Logistic Regression)", accuracy_score(y_train, bert_model_logreg.predict(X_train_bert)) * 100),
]

test_results = [
    ("TF-IDF (Logistic Regression)", accuracy_score(df_test_hidden['sentiment_label'], tfidf_model.predict(X_test_tfidf)) * 100),
    ("Word2Vec (SVM)", accuracy_score(df_test_hidden['sentiment_label'], w2v_model_svm.predict(X_test_w2v)) * 100),
    ("BERT (Logistic Regression)", accuracy_score(df_test_hidden['sentiment_label'], bert_model_logreg.predict(X_test_bert)) * 100),
]

# Convert to DataFrames
train_results_df = pd.DataFrame(train_results, columns=["Model", "Train Accuracy (%)"])
test_results_df = pd.DataFrame(test_results, columns=["Model", "Test Accuracy (%)"])

# Print results in a structured format
print("\nTrain Accuracy Comparison:\n")
print(train_results_df.to_string(index=False))

print("\nTest Accuracy Comparison (Hidden Data):\n")
print(test_results_df.to_string(index=False))

# Save results to CSV files for reference
train_results_path = "./dataset/train_accuracy_results.csv"
test_results_path = "./dataset/test_accuracy_results.csv"

train_results_df.to_csv(train_results_path, index=False)
test_results_df.to_csv(test_results_path, index=False)

# Provide download paths
train_results_path, test_results_path



Train Accuracy Comparison:

                       Model  Train Accuracy (%)
TF-IDF (Logistic Regression)              94.050
              Word2Vec (SVM)              93.725
  BERT (Logistic Regression)              97.375

Test Accuracy Comparison (Hidden Data):

                       Model  Test Accuracy (%)
TF-IDF (Logistic Regression)               93.8
              Word2Vec (SVM)               93.7
  BERT (Logistic Regression)               94.3


('./dataset/train_accuracy_results.csv', './dataset/test_accuracy_results.csv')